In [2]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery


In [13]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados novos\\Dados para subir\\Dados Brutos\\ENAP - ações afirmativas')


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   universidade          63 non-null     object
 1    quantidade_cotista   63 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [41]:
df = pd.read_csv('perc_professor_superior.csv', sep=';', decimal=',', encoding='UTF-8')
df

,sigla,nome_universidade,quantidade_cotista
0,FUAM,Fundação Universidade Federal do Amazonas,-
1,FUFOP,Fundação Universidade Federal de Ouro Preto,-
2,FUFPEL,Fundação Universidade Federal de Pelotas,-
3,FUFSCAR,Universidade Federal de São Carlos - SP,-
4,FUMA,Fundação Universidade Federal do Maranhão,-
...,...,...,...
59,UFSC,Universidade Federal de Santa Catarina,"0,0197"
60,UFOPA,Universidade Federal do Oeste do Pará,"0,0238"
61,UFMS,Universidade Federal de Mato Grosso do Sul,"0,0370"
62,UFGD,Fundação Universidade Federal da Grande Dourados,"0,0428"


In [42]:
df = df[df['sigla']!="FUMA"]

Removi FUMA pois é um nome antigo da Universidade Federal do Maranhão, a qual já está presente na lista. 
 Mais detalhes podem ser encontrados no site:  https://portalpadrao.ufma.br/pi/historico-institucional

In [43]:
df = df.rename(columns={' quantidade_cotista ':'per_cotistas'})


In [29]:
df.columns

Index(['sigla', 'nome_universidade', 'per_cotistas'], dtype='object')

In [44]:
df['per_cotistas'] = df['per_cotistas'].str.replace(',','.').str.replace('-','0').astype(float)

In [40]:
df

,sigla,nome_universidade,per_cotistas
63,UFESBA,Universidade Federal do Sul da Bahia,0.0774
62,UFGD,Fundação Universidade Federal da Grande Dourados,0.0428
61,UFMS,Universidade Federal de Mato Grosso do Sul,0.0370
60,UFOPA,Universidade Federal do Oeste do Pará,0.0238
59,UFSC,Universidade Federal de Santa Catarina,0.0197
58,FUFT,Fundação Universidade Federal do Tocantins,0.0150
57,UNIR,Fundação Universidade Federal de Rondônia,0.0144
56,UFBA,Universidade Federal da Bahia,0.0131
55,UFLA,Universidade Federal de Lavras - MG,0.0130
54,UFMT,Universidade Federal de Mato Grosso,0.0128


In [45]:
df = df.sort_values(by=['per_cotistas'], ascending=False)

In [46]:
len(df)

63

Subindo para o GBQ

In [51]:
client = bigquery.Client()
dataset_ref = client.dataset('acoes_afirmativas')

In [52]:
schema = [bigquery.SchemaField('universidade', 'STRING', description= 'Universidades observadas'),
         bigquery.SchemaField('per_cotistas', 'FLOAT', description= 'Percentual de cotistas observados')]
         

In [53]:
table_ref = dataset_ref.table('SIAPE_perc_professor_superior')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=225000b5-4442-4aa5-9bbd-12150d36bb86>